In [ ]:
from azureml.core import Workspace, Experiment

subscription_id = 'a022d83d-6229-4b5e-b039-b680692436b5'
resource_group = 'azureai-test-east'
workspace_name = 'project-azureai-patton-test-east'

ws = Workspace(subscription_id, resource_group, workspace_name)
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "udacity_cluster"

try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling( {  
    "--C": choice(0.001, 0.01, 0.1, 1, 10, 20, 50, 100, 200, 500, 1000),
    "--max_iter": choice(100, 200, 300)
    }
)

# Specify a Policy: check the job every 2 iterations and if the primary metric falls outside the top 50% range, 
# Azure ML should terminate the job
policy = BanditPolicy(evaluation_interval=5, slack_factor=0.1)


if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ### YOUR CODE HERE ###
est = SKLearn(source_directory = "./", compute_target=cpu_cluster, vm_size="STANDARD_DS3_V2", entry_script="train.py")

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.

hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                    primary_metric_name="Accuracy", 
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    policy=policy,
                                    estimator=est,
                                    max_total_runs=10, 
                                    max_concurrent_runs=5)




In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hypdrive_run = exp.submit(hyperdrive_config)

# Show run details
RunDetails(hypdrive_run).show()

In [ ]:
import joblib
# Get your best run and save the model from that run.

hd_best_run = hypdrive_run.get_best_run_by_primary_metric()
hd_best_model = hd_best_run.register_model(model_name="hd_best_model" , model_path="./")

best_run_metrics = hd_best_run.get_metrics()
parameter_values = hd_best_run.get_details()

print("Best Run Id: ", hd_best_run.id)
print("Best run metrics: ", best_run_metrics)
print("Accuracy: ", best_run_metrics["Accuracy"])
print("Best run details ", parameter_values)
print("Parameters: ", parameter_values["runDefinition"]["arguments"])

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

csv_url = 'https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv'    

# Create a TabularDataset using TabularDatasetFactory
ds = TabularDatasetFactory.from_delimited_files(path=csv_url)

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task=,
    primary_metric=,
    training_data=,
    label_column_name=,
    n_cross_validations=)

In [2]:
# Submit your automl run

remote_run = exp.submit(automl_config, show_output=False)
remote_run.wait_for_completion()

In [ ]:
# Retrieve and save your best automl model.

best_run, fitted_model = remote_run.get_output()

print(best_run)
print(fitted_model)

print("Best run metrics: ", best_run.get_metrics())
print("Best run details: ", best_run.get_details())